In [1]:
import pandas as pd
import numpy as np
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import gc
import h5py

from PIL import Image
from pyts.image import RecurrencePlot, GramianAngularField
from pyts.datasets import load_gunpoint
from pyts.image import MarkovTransitionField
from pyts.image import GramianAngularField

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import tensorflow.python.keras.backend as K

from keras import optimizers
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Flatten, Conv3D,Conv2D, MaxPooling3D, MaxPooling2D, Dropout, LSTM
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
from keras.applications import VGG16
from keras.optimizers import Adam

Using TensorFlow backend.


# Make Graph

In [ ]:
file_list = os.listdir('input')
save_path = 'E:/Data/new_EEG/graph/7680/'
for file in file_list:
    data = pd.read_csv('input/'+file, header=0)
    
    for i in range(16):
        plt.plot(data.iloc[:,i])
        plt.axis('off')       
        plt.savefig(save_path+file[:-4]+'_%d.jpg' %(i+1), dpi=100)
        plt.cla()

file_list = os.listdir('input_split_6')
save_path = 'E:/Data/new_EEG/graph/1280/'
for file in file_list:
    data = pd.read_csv('input_split_6/'+file, header=0)
    
    for i in range(16):
        plt.plot(data.iloc[:,i])
        plt.axis('off')       
        plt.savefig(save_path+file[:-4]+'_%d.jpg' %(i+1), dpi=100)
        plt.cla()

file_list = os.listdir('input_split_12')
save_path = 'E:/Data/new_EEG/graph/640/'
for file in file_list:
    data = pd.read_csv('input_split_12/'+file, header=0)
    
    for i in range(16):
        plt.plot(data.iloc[:,i])
        plt.axis('off')       
        plt.savefig(save_path+file[:-4]+'_%d.jpg' %(i+1), dpi=100)
        plt.cla()

In [1]:
# Learning Curve

def draw_plot(hist):
    fig, loss_ax = plt.subplots()

    acc_ax = loss_ax.twinx()

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

    acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')

    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    acc_ax.set_ylabel('accuray')

    loss_ax.legend(loc='upper left')
    acc_ax.legend(loc='lower left')

    plt.show()

# Model

In [3]:
def create_model():
    pre_trained_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    pre_trained_vgg.trainable = False

    model = Sequential()
    model.add(pre_trained_vgg)
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.8))
    #model.add(Dropout(rate=0.2))
    model.add(Dense(2048, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(0.5))
    #model.add(Dense(256, activation='relu'))
    #model.add(Dense(64, activation='relu'))
    #model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
    
    #return model
    return model, model.get_weights()


# Make Input Data & Label

In [4]:
path = 'E:/Data/new_EEG/vgg_data_graph/7680/data/'
x_data = []
y_data = []
file_list = os.listdir(path)

for file in file_list:
    if file[:3] == 'sch':
        y_data.append(1)
    else:
        y_data.append(0)
        
    data = np.load(path+file)
    data = data['arr_0']
    x_data.append(data)
        
x_data = np.array(x_data)
y_data = np.array(y_data)

In [5]:
x_data.shape

(1344, 224, 224, 3)

In [7]:
kfold = KFold(n_splits=10, shuffle=True, random_state=1004)

In [8]:
def init_weight(same_old_model, first_weights):
    same_old_model.set_weights(weights)

In [ ]:
def cf_mat(test):
    y_pred = model.predict_classes(x_data[test])
    return confusion_matrix(y_data[test], y_pred)

In [9]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=50, restore_best_weights=True)

# Train & Test

In [10]:
path = 'E:/Data/new_EEG/vgg_data_graph/7680/'
num = 0
k_accuracy = list()
accuracy = list()

model, weights = create_model()
for train, test in kfold.split(x_data):

    num = num+1
    print("---------------------------epoch : ",num,"---------------------------")
    init_weight(model, weights)
    model.fit(x_data[train], y_data[train], epochs=200, batch_size=20, callbacks=[es])
    accuracy =  (model.evaluate(x_data[test], y_data[test]))
    k_accuracy.append(accuracy)
    
    mat.append(cf_mat(test))

---------------------------epoch :  1 ---------------------------
Epoch 1/200
1209/1209 [==============================] - 11s 9ms/step - loss: 0.9488 - accuracy: 0.5244
Epoch 2/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.8559 - accuracy: 0.5294
Epoch 3/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.7678 - accuracy: 0.5434
Epoch 4/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.7487 - accuracy: 0.5484
Epoch 5/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.7194 - accuracy: 0.5558
Epoch 6/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.7272 - accuracy: 0.5376
Epoch 7/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.6695 - accuracy: 0.5848
Epoch 8/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.6828 - accuracy: 0.5881
Epoch 9/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.6575 - accuracy: 0.6129
Epoch 

1209/1209 [==============================] - 7s 6ms/step - loss: 0.4743 - accuracy: 0.7601
Epoch 156/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4839 - accuracy: 0.7601
Epoch 157/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4669 - accuracy: 0.7758
Epoch 158/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4796 - accuracy: 0.7452
Epoch 159/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4634 - accuracy: 0.7775
Epoch 160/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4608 - accuracy: 0.7758
Epoch 161/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4806 - accuracy: 0.7585
Epoch 162/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4781 - accuracy: 0.7519
Epoch 163/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4858 - accuracy: 0.7568
Epoch 164/200
1209/1209 [==============================] - 7s 6ms/ste

1209/1209 [==============================] - 7s 6ms/step - loss: 0.4830 - accuracy: 0.7270
Epoch 111/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4624 - accuracy: 0.7552
Epoch 112/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4656 - accuracy: 0.7692
Epoch 113/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4604 - accuracy: 0.7626
Epoch 114/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4652 - accuracy: 0.7577
Epoch 115/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4691 - accuracy: 0.7494
Epoch 116/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4710 - accuracy: 0.7494
Epoch 117/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4636 - accuracy: 0.7667
Epoch 118/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4479 - accuracy: 0.7684
Epoch 119/200
1209/1209 [==============================] - 7s 6ms/ste

1209/1209 [==============================] - 7s 6ms/step - loss: 0.4285 - accuracy: 0.7461
Epoch 123/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4177 - accuracy: 0.7783
Epoch 124/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4107 - accuracy: 0.7709
Epoch 125/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4257 - accuracy: 0.7585
Epoch 126/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4327 - accuracy: 0.7717
Epoch 127/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4342 - accuracy: 0.7734
Epoch 128/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4164 - accuracy: 0.7899
Epoch 129/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4156 - accuracy: 0.7883
Epoch 130/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4180 - accuracy: 0.7767
Epoch 131/200
1209/1209 [==============================] - 7s 6ms/ste

1209/1209 [==============================] - 7s 6ms/step - loss: 0.4491 - accuracy: 0.7808
Epoch 78/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4436 - accuracy: 0.7758
Epoch 79/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4603 - accuracy: 0.7684
Epoch 80/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4311 - accuracy: 0.7816
Epoch 81/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4487 - accuracy: 0.7940
Epoch 82/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4422 - accuracy: 0.7866
Epoch 83/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4474 - accuracy: 0.7742
Epoch 84/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4286 - accuracy: 0.7841
Epoch 85/200
1209/1209 [==============================] - 7s 6ms/step - loss: 0.4448 - accuracy: 0.7552
Epoch 86/200
1209/1209 [==============================] - 7s 6ms/step - loss:

...

1210/1210 [==============================] - 7s 6ms/step - loss: 0.5774 - accuracy: 0.6983
Epoch 33/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.5675 - accuracy: 0.7066
Epoch 34/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.5582 - accuracy: 0.7149
Epoch 35/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.5639 - accuracy: 0.7025
Epoch 36/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.5485 - accuracy: 0.7157
Epoch 37/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.5634 - accuracy: 0.6942
Epoch 38/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.5759 - accuracy: 0.6884
Epoch 39/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.5434 - accuracy: 0.7083
Epoch 40/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.5379 - accuracy: 0.7248
Epoch 41/200
1210/1210 [==============================] - 7s 6ms/step - loss:

1210/1210 [==============================] - 7s 6ms/step - loss: 0.3889 - accuracy: 0.7818
Epoch 189/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4138 - accuracy: 0.7835
Epoch 190/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4142 - accuracy: 0.7785
Epoch 191/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4306 - accuracy: 0.7818
Epoch 192/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.3882 - accuracy: 0.8124
Epoch 193/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4132 - accuracy: 0.7868
Epoch 194/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4132 - accuracy: 0.7893
Epoch 195/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4610 - accuracy: 0.7314
Epoch 196/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4096 - accuracy: 0.7868
Epoch 197/200
1210/1210 [==============================] - 7s 6ms/ste

1210/1210 [==============================] - 7s 6ms/step - loss: 0.4556 - accuracy: 0.7636
Epoch 144/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4673 - accuracy: 0.7579
Epoch 145/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4648 - accuracy: 0.7612
Epoch 146/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4579 - accuracy: 0.7603
Epoch 147/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4857 - accuracy: 0.7256
Epoch 148/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4568 - accuracy: 0.7636
Epoch 149/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4769 - accuracy: 0.7463
Epoch 150/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4662 - accuracy: 0.7479
Epoch 151/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4701 - accuracy: 0.7455
Epoch 152/200
1210/1210 [==============================] - 7s 6ms/ste

1210/1210 [==============================] - 7s 6ms/step - loss: 0.4476 - accuracy: 0.7769
Epoch 99/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4334 - accuracy: 0.7736
Epoch 100/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4620 - accuracy: 0.7364
Epoch 101/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4638 - accuracy: 0.7496
Epoch 102/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4341 - accuracy: 0.7727
Epoch 103/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4539 - accuracy: 0.7587
Epoch 104/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4154 - accuracy: 0.7901
Epoch 105/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4381 - accuracy: 0.7826
Epoch 106/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4332 - accuracy: 0.7810
Epoch 107/200
1210/1210 [==============================] - 7s 6ms/step

1210/1210 [==============================] - 7s 6ms/step - loss: 0.5067 - accuracy: 0.7562
Epoch 54/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.5039 - accuracy: 0.7380
Epoch 55/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4886 - accuracy: 0.7595
Epoch 56/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.5117 - accuracy: 0.7471
Epoch 57/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4967 - accuracy: 0.7488
Epoch 58/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.5036 - accuracy: 0.7421
Epoch 59/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.5018 - accuracy: 0.7554
Epoch 60/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4813 - accuracy: 0.7545
Epoch 61/200
1210/1210 [==============================] - 7s 6ms/step - loss: 0.4925 - accuracy: 0.7438
Epoch 62/200
1210/1210 [==============================] - 7s 6ms/step - loss:

In [11]:
k_accuracy

[[0.5761527692830121, 0.6518518328666687],
 [0.5152462836768892, 0.7407407164573669],
 [0.7242076873779297, 0.6518518328666687],
 [0.5683538446823756, 0.7111111283302307],
 [0.5590443353154766, 0.7014925479888916],
 [0.5030941800839865, 0.7388059496879578],
 [0.5437019547419761, 0.7910447716712952],
 [0.5466317165253768, 0.746268630027771],
 [0.5996167744273571, 0.7238805890083313],
 [0.5587691686936279, 0.7238805890083313]]

In [ ]:
mat